In [113]:
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import HuggingFaceDatasetLoader, YoutubeLoader
from langchain_community.chat_models import ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from langchain.vectorstores import Pinecone
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
from openai import OpenAI
import tiktoken
import numpy as np
import os

load_dotenv()

OPEN_ROUTER_API_KEY = os.getenv("OPEN_ROUTER_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [114]:
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/bert-large-nli-max-tokens")
vectorstore = PineconeVectorStore(index_name="customer-support", embedding=hf_embeddings)
vectorstore = vectorstore.from_existing_index(index_name="customer-support",
                                           embedding=hf_embeddings)

retriever = vectorstore.as_retriever()

/home/user/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [115]:
# RAG
openrouter_client = ChatOpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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"
)

In [116]:
vectorstore = PineconeVectorStore(index_name="customer-support", embedding=hf_embeddings)

index_name = 'customer-support'
namespace = "langchain-docs"

In [117]:
template = """Answer the question based only on the following context:
{context}
Question: {question}"""

prompt = ChatPromptTemplate.from_template(template)

In [118]:
retreiver = vectorstore.as_retriever()

chain = (
  RunnableParallel({"context": retreiver, "question": RunnablePassthrough()})
  | prompt
  | openrouter_client
  | StrOutputParser()
)

In [119]:
chain.invoke("Tell me about jonas")

'Jonas is someone who has a deep understanding of happiness and the pursuit of meaning in life. He emphasizes the importance of living a life according to love and spiritual fullness rather than solely pursuing happiness. Jonas also shares a personal experience of giving up certain things in pursuit of his purpose, only to find unexpected fulfillment and success in areas he had not anticipated. He also has a family history marked by suffering, including surviving the Armenian and Assyrian genocides and the Nazi occupation of France, which seems to have influenced his views on life and purpose.'